In [1]:
import json
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math

In [2]:
od_data = np.load('od_matrix.npy')
duration_data = np.load('duration_matrix.npy')
distance_data = np.load('distance_matrix.npy')
charging_stations = np.load('charging_stations.npy')
demand_per_time = np.zeros(od_data.shape[2])
for t in range(od_data.shape[2]):
    demand_per_time[t] = od_data[:,:,t].sum()
low_demand_times = demand_per_time < demand_per_time.max()*0.7
od_data = od_data[:, :, 8:-4] # in requests
duration_data = duration_data[:, :, 8:-4] # in sec
distance_data = distance_data[:, :, 8:-4] # in miles
print(od_data.shape)
print(od_data.sum())

# set remaing nans to avg
avg_duration = int(np.nanmean(duration_data))
duration_data[np.isnan(duration_data)] = avg_duration
avg_distance = int(np.nanmean(distance_data))
distance_data[np.isnan(distance_data)] = avg_distance
delta_c = 1.5 # energy step [kWh] 0.75
time_granularity = 0.25 # in h

chevy_bolt_capacity = 65 # in kWh
chevy_bolt_usable_capacity = chevy_bolt_capacity * 0.6 # never go below 20% or above 80% of charge
charger_capacity = 50.0 # assuming 50. kW Chargers
assert (delta_c/charger_capacity)/time_granularity < 1
charge_levels_per_charge_step = math.floor((charger_capacity*time_granularity)/delta_c)
# print(charge_levels_per_charge_step)
chevy_bolt_range = 230 # range in mi for mild city trips according to https://media.chevrolet.com/media/us/en/chevrolet/2022-bolt-euv-bolt-ev.detail.html/content/Pages/news/us/en/2021/feb/0214-boltev-bolteuv-specifications.html
chevy_bolt_usable_range = chevy_bolt_range*0.6 # never go below 20% or above 80% of charge and assume 10% less efficient because of range https://cleantechnica.com/2017/10/13/autonomous-cars-shorter-range-due-high-power-consumption-computers/
chevy_bolt_kwh_per_mi = (chevy_bolt_usable_capacity/chevy_bolt_usable_range)/0.7 # we lose 30% efficiency because of AVs (https://medium.com/@teraki/energy-consumption-required-by-edge-computing-reduces-a-autonomous-cars-mileage-with-up-to-30-46b6764ea1b7)
# print((distance_data * chevy_bolt_kwh_per_mi)/delta_c)
# print(chevy_bolt_kwh_per_mi)
energy_distance = np.ceil(((distance_data * chevy_bolt_kwh_per_mi)/delta_c).max(axis=2))
energy_distance[energy_distance==0] = 1 # we should always use energy to satisfy a trip
print(energy_distance)
np.save('energy_distance.npy', energy_distance)
# print(np.sum(energy_distance==1.))
# print(energy_distance.max())
duration_data = np.round(duration_data/(3600*time_granularity)) # convert travel time from sec to h
duration_data[duration_data==0] = 1. # it should always take time to satisfy a trip
data_timespan = od_data.shape[2]
episode_length = int(data_timespan/time_granularity)
# https://www.pge.com/tariffs/assets/pdf/tariffbook/ELEC_SCHEDS_BEV.pdf for prices of energy
p_energy = np.ones(episode_length)*0.16872 # in $/kWh
p_energy[int(16/time_granularity):int(21/time_granularity)] = 0.38195 # peak prices
p_energy[int(9/time_granularity):int(14/time_granularity)] = 0.14545 # super off peak prices
p_energy *= delta_c # in $/ charge level
p_travel = 0.0770 # [$ / mi] https://newsroom.aaa.com/wp-content/uploads/2021/08/2021-YDC-Brochure-Live.pdf 
avg_miles_per_h_driving = 9.6 # in m/h from 190x190x24
operational_cost_per_timestep = avg_miles_per_h_driving * time_granularity * p_travel
print(operational_cost_per_timestep)
fleet_size = 116616 # got number from Justin: 116616

number_chargelevels = int(chevy_bolt_usable_capacity/delta_c)
number_spatial_nodes = 5
charge_locations = np.ones(number_spatial_nodes,dtype=bool).tolist()
cars_per_station_capacity = charging_stations.tolist()

print(charge_levels_per_charge_step)
print(number_chargelevels)

(5, 5, 12)
1743156
[[1. 2. 3. 2. 2.]
 [2. 1. 2. 2. 3.]
 [3. 2. 1. 2. 2.]
 [2. 2. 2. 1. 2.]
 [2. 3. 2. 2. 1.]]
0.1848
8
26


In [3]:
new_tripAttr = []
new_reb_time = []
new_total_acc = []
temp_demand = 0
for origin in range(duration_data.shape[0]):
    for destination in range(duration_data.shape[1]):
        for ts in range(episode_length):
            attr = defaultdict()
            attr['time_stamp'] = ts
            attr['origin'] = origin
            attr['destination'] = destination
            attr['demand'] = round(od_data[origin,destination,int(ts*time_granularity)]*time_granularity) # create equal distributed demand over granular time 
            temp_demand += attr['demand']
            attr['price'] = distance_data[origin,destination,int(ts*time_granularity)]*0.91 + duration_data[origin,destination,int(ts*time_granularity)]*time_granularity * 0.39 + 2.20 + 2.70 # in $
            new_tripAttr.append(attr)

            reb = defaultdict()
            reb['time_stamp'] = ts
            reb['origin'] = origin
            reb['destination'] = destination
            reb['reb_time'] = int(duration_data[origin,destination,int(ts*time_granularity)])
            new_reb_time.append(reb)
print(temp_demand)
# should be able to change 24 to 1, we don't change the number of vehicles across the day
for hour in range(24):
    acc = defaultdict()
    acc['hour'] = hour
    acc['acc'] = math.ceil(fleet_size)
    new_total_acc.append(acc)
new_data = defaultdict()
new_data['demand'] = new_tripAttr
new_data['rebTime'] = new_reb_time
new_data['totalAcc'] = new_total_acc
new_data['chargelevels'] = number_chargelevels
new_data['spatialNodes'] = number_spatial_nodes
new_data['chargeLevelsPerChargeStep'] = charge_levels_per_charge_step
new_data['episodeLength'] = episode_length
new_data['energy_prices'] = p_energy.tolist()
new_data['chargeLocations'] = charge_locations
new_data['carsPerStationCapacity'] = cars_per_station_capacity
new_data['timeGranularity'] = time_granularity
new_data['operationalCostPerTimestep'] = operational_cost_per_timestep
print(episode_length)
with open('SF_5.json', 'w') as f:
    json.dump(new_data, f)

1743164
48


In [4]:
new_tripAttr = []
new_reb_time = []
new_total_acc = []
temp_demand = 0
for origin in range(duration_data.shape[0]):
    for destination in range(duration_data.shape[1]):
        for ts in range(episode_length):
            attr = defaultdict()
            attr['time_stamp'] = ts
            attr['origin'] = origin
            attr['destination'] = destination
            attr['demand'] = round(0.95*od_data[origin,destination,int(ts*time_granularity)]*time_granularity) # create equal distributed demand over granular time 
            temp_demand += attr['demand']
            attr['price'] = distance_data[origin,destination,int(ts*time_granularity)]*0.91 + duration_data[origin,destination,int(ts*time_granularity)]*time_granularity * 0.39 + 2.20 + 2.70 # in $
            new_tripAttr.append(attr)

            reb = defaultdict()
            reb['time_stamp'] = ts
            reb['origin'] = origin
            reb['destination'] = destination
            reb['reb_time'] = int(duration_data[origin,destination,int(ts*time_granularity)])
            new_reb_time.append(reb)
print(temp_demand)
# should be able to change 24 to 1, we don't change the number of vehicles across the day
for hour in range(24):
    acc = defaultdict()
    acc['hour'] = hour
    acc['acc'] = math.ceil(fleet_size)
    new_total_acc.append(acc)
new_data = defaultdict()
new_data['demand'] = new_tripAttr
new_data['rebTime'] = new_reb_time
new_data['totalAcc'] = new_total_acc
new_data['chargelevels'] = number_chargelevels
new_data['spatialNodes'] = number_spatial_nodes
new_data['chargeLevelsPerChargeStep'] = charge_levels_per_charge_step
new_data['episodeLength'] = episode_length
new_data['energy_prices'] = p_energy.tolist()
new_data['chargeLocations'] = charge_locations
new_data['carsPerStationCapacity'] = cars_per_station_capacity
new_data['timeGranularity'] = time_granularity
new_data['operationalCostPerTimestep'] = operational_cost_per_timestep
print(episode_length)
with open('SF_5_test.json', 'w') as f:
    json.dump(new_data, f)

1656008
48


In [4]:
# preprocessing for coarser time
od_data = np.load('od_matrix.npy')
duration_data = np.load('duration_matrix.npy')
distance_data = np.load('distance_matrix.npy')
charging_stations = np.load('charging_stations.npy')
demand_per_time = np.zeros(od_data.shape[2])
for t in range(od_data.shape[2]):
    demand_per_time[t] = od_data[:,:,t].sum()
low_demand_times = demand_per_time < demand_per_time.max()*0.7
od_data = od_data[:, :, 8:-4] # in requests
duration_data = duration_data[:, :, 8:-4] # in sec
distance_data = distance_data[:, :, 8:-4] # in miles
print(od_data.shape)
print(od_data.sum())

# set remaing nans to avg
avg_duration = int(np.nanmean(duration_data))
duration_data[np.isnan(duration_data)] = avg_duration
avg_distance = int(np.nanmean(distance_data))
distance_data[np.isnan(distance_data)] = avg_distance
delta_c = 1.5 # energy step [kWh] 0.75
time_granularity = 0.5 # in h

chevy_bolt_capacity = 65 # in kWh
chevy_bolt_usable_capacity = chevy_bolt_capacity * 0.6 # never go below 20% or above 80% of charge
charger_capacity = 50.0 # assuming 50. kW Chargers
assert (delta_c/charger_capacity)/time_granularity < 1
charge_levels_per_charge_step = math.floor((charger_capacity*time_granularity)/delta_c)
# print(charge_levels_per_charge_step)
chevy_bolt_range = 230 # range in mi for mild city trips according to https://media.chevrolet.com/media/us/en/chevrolet/2022-bolt-euv-bolt-ev.detail.html/content/Pages/news/us/en/2021/feb/0214-boltev-bolteuv-specifications.html
chevy_bolt_usable_range = chevy_bolt_range*0.6 # never go below 20% or above 80% of charge and assume 10% less efficient because of range https://cleantechnica.com/2017/10/13/autonomous-cars-shorter-range-due-high-power-consumption-computers/
chevy_bolt_kwh_per_mi = (chevy_bolt_usable_capacity/chevy_bolt_usable_range)/0.7 # we lose 30% efficiency because of AVs (https://medium.com/@teraki/energy-consumption-required-by-edge-computing-reduces-a-autonomous-cars-mileage-with-up-to-30-46b6764ea1b7)
# print((distance_data * chevy_bolt_kwh_per_mi)/delta_c)
# print(chevy_bolt_kwh_per_mi)
energy_distance = np.ceil(((distance_data * chevy_bolt_kwh_per_mi)/delta_c).max(axis=2))
energy_distance[energy_distance==0] = 1 # we should always use energy to satisfy a trip
print(energy_distance)
np.save('energy_distance.npy', energy_distance)
# print(np.sum(energy_distance==1.))
# print(energy_distance.max())
duration_data = np.round(duration_data/(3600*time_granularity)) # convert travel time from sec to h
duration_data[duration_data==0] = 1. # it should always take time to satisfy a trip
data_timespan = od_data.shape[2]
episode_length = int(data_timespan/time_granularity)
# https://www.pge.com/tariffs/assets/pdf/tariffbook/ELEC_SCHEDS_BEV.pdf for prices of energy
p_energy = np.ones(episode_length)*0.16872 # in $/kWh
p_energy[int(16/time_granularity):int(21/time_granularity)] = 0.38195 # peak prices
p_energy[int(9/time_granularity):int(14/time_granularity)] = 0.14545 # super off peak prices
p_energy *= delta_c # in $/ charge level
p_travel = 0.0770 # [$ / mi] https://newsroom.aaa.com/wp-content/uploads/2021/08/2021-YDC-Brochure-Live.pdf 
avg_miles_per_h_driving = 9.6 # in m/h from 190x190x24
operational_cost_per_timestep = avg_miles_per_h_driving * time_granularity * p_travel
print(operational_cost_per_timestep)
fleet_size = 116616*1.5 # got number from Justin: 116616

number_chargelevels = int(chevy_bolt_usable_capacity/delta_c)
number_spatial_nodes = 5
charge_locations = np.ones(number_spatial_nodes,dtype=bool).tolist()
print("number chargers:", sum(charging_stations))
cars_per_station_capacity = charging_stations.tolist()

print(charge_levels_per_charge_step)
print(number_chargelevels)

(5, 5, 12)
1743156
[[1. 2. 3. 2. 2.]
 [2. 1. 2. 2. 3.]
 [3. 2. 1. 2. 2.]
 [2. 2. 2. 1. 2.]
 [2. 3. 2. 2. 1.]]
0.3696
number chargers: 5906.0
16
26


In [5]:
new_tripAttr = []
new_reb_time = []
new_total_acc = []
temp_demand = 0
for origin in range(duration_data.shape[0]):
    for destination in range(duration_data.shape[1]):
        for ts in range(episode_length):
            attr = defaultdict()
            attr['time_stamp'] = ts
            attr['origin'] = origin
            attr['destination'] = destination
            attr['demand'] = round(od_data[origin,destination,int(ts*time_granularity)]*time_granularity) # create equal distributed demand over granular time 
            temp_demand += attr['demand']
            attr['price'] = distance_data[origin,destination,int(ts*time_granularity)]*0.91 + duration_data[origin,destination,int(ts*time_granularity)]*time_granularity * 0.39 + 2.20 + 2.70 # in $
            new_tripAttr.append(attr)

            reb = defaultdict()
            reb['time_stamp'] = ts
            reb['origin'] = origin
            reb['destination'] = destination
            reb['reb_time'] = int(duration_data[origin,destination,int(ts*time_granularity)])
            new_reb_time.append(reb)
print(temp_demand)
# should be able to change 24 to 1, we don't change the number of vehicles across the day
for hour in range(24):
    acc = defaultdict()
    acc['hour'] = hour
    acc['acc'] = math.ceil(fleet_size)
    new_total_acc.append(acc)
new_data = defaultdict()
new_data['demand'] = new_tripAttr
new_data['rebTime'] = new_reb_time
new_data['totalAcc'] = new_total_acc
new_data['chargelevels'] = number_chargelevels
new_data['spatialNodes'] = number_spatial_nodes
new_data['chargeLevelsPerChargeStep'] = charge_levels_per_charge_step
new_data['episodeLength'] = episode_length
new_data['energy_prices'] = p_energy.tolist()
new_data['chargeLocations'] = charge_locations
new_data['carsPerStationCapacity'] = cars_per_station_capacity
new_data['timeGranularity'] = time_granularity
new_data['operationalCostPerTimestep'] = operational_cost_per_timestep
print(episode_length)
with open('SF_5_short.json', 'w') as f:
    json.dump(new_data, f)

1743154
24


In [3]:
# preprocessing for coarser time afternoon
od_data = np.load('od_matrix.npy')
duration_data = np.load('duration_matrix.npy')
distance_data = np.load('distance_matrix.npy')
charging_stations = np.load('charging_stations.npy')
demand_per_time = np.zeros(od_data.shape[2])
for t in range(od_data.shape[2]):
    demand_per_time[t] = od_data[:,:,t].sum()
low_demand_times = demand_per_time < demand_per_time.max()*0.7
print(low_demand_times)
od_data = od_data[:, :, 12:-4] # in requests
duration_data = duration_data[:, :, 12:-4] # in sec
distance_data = distance_data[:, :, 12:-4] # in miles
print(od_data.shape)
print(od_data.sum())

# set remaing nans to avg
avg_duration = int(np.nanmean(duration_data))
duration_data[np.isnan(duration_data)] = avg_duration
avg_distance = int(np.nanmean(distance_data))
distance_data[np.isnan(distance_data)] = avg_distance
delta_c = 1.5 # energy step [kWh] 0.75
time_granularity = 0.5 # in h

chevy_bolt_capacity = 65 # in kWh
chevy_bolt_usable_capacity = chevy_bolt_capacity * 0.6 # never go below 20% or above 80% of charge
charger_capacity = 50.0 # assuming 50. kW Chargers
assert (delta_c/charger_capacity)/time_granularity < 1
charge_levels_per_charge_step = math.floor((charger_capacity*time_granularity)/delta_c)
# print(charge_levels_per_charge_step)
chevy_bolt_range = 230 # range in mi for mild city trips according to https://media.chevrolet.com/media/us/en/chevrolet/2022-bolt-euv-bolt-ev.detail.html/content/Pages/news/us/en/2021/feb/0214-boltev-bolteuv-specifications.html
chevy_bolt_usable_range = chevy_bolt_range*0.6 # never go below 20% or above 80% of charge and assume 10% less efficient because of range https://cleantechnica.com/2017/10/13/autonomous-cars-shorter-range-due-high-power-consumption-computers/
chevy_bolt_kwh_per_mi = (chevy_bolt_usable_capacity/chevy_bolt_usable_range)/0.7 # we lose 30% efficiency because of AVs (https://medium.com/@teraki/energy-consumption-required-by-edge-computing-reduces-a-autonomous-cars-mileage-with-up-to-30-46b6764ea1b7)
print("distance per charge level", delta_c/chevy_bolt_kwh_per_mi)
# print(chevy_bolt_kwh_per_mi)
energy_distance = np.ceil(((distance_data * chevy_bolt_kwh_per_mi)/delta_c).max(axis=2))
energy_distance[energy_distance==0] = 1 # we should always use energy to satisfy a trip
print(energy_distance)
np.save('energy_distance.npy', energy_distance)
# print(np.sum(energy_distance==1.))
# print(energy_distance.max())
duration_data = np.round(duration_data/(3600*time_granularity)) # convert travel time from sec to h
duration_data[duration_data==0] = 1. # it should always take time to satisfy a trip
data_timespan = od_data.shape[2]
episode_length = int(data_timespan/time_granularity)
# https://www.pge.com/tariffs/assets/pdf/tariffbook/ELEC_SCHEDS_BEV.pdf for prices of energy
p_energy = np.ones(episode_length)*0.16872 # in $/kWh
p_energy[int(16/time_granularity):int(21/time_granularity)] = 0.38195 # peak prices
p_energy[int(9/time_granularity):int(14/time_granularity)] = 0.14545 # super off peak prices
p_energy *= delta_c # in $/ charge level
p_travel = 0.0770 # [$ / mi] https://newsroom.aaa.com/wp-content/uploads/2021/08/2021-YDC-Brochure-Live.pdf 
avg_miles_per_h_driving = 9.6 # in m/h from 190x190x24
operational_cost_per_timestep = avg_miles_per_h_driving * time_granularity * p_travel
print(operational_cost_per_timestep)
fleet_size = 116616*1.5 # got number from Justin: 116616

number_chargelevels = int(chevy_bolt_usable_capacity/delta_c)
number_spatial_nodes = 5
charge_locations = np.ones(number_spatial_nodes,dtype=bool).tolist()
cars_per_station_capacity = charging_stations.tolist()

print(charge_levels_per_charge_step)
print(number_chargelevels)

[ True  True  True  True  True  True  True False False  True  True  True
  True  True False False False False False  True  True  True  True  True]
(5, 5, 8)
1228031
distance per charge level 3.7153846153846155
[[1. 2. 3. 2. 2.]
 [2. 1. 2. 2. 3.]
 [3. 2. 1. 2. 2.]
 [2. 2. 2. 1. 2.]
 [2. 3. 2. 2. 1.]]
0.3696
16
26


In [3]:
new_tripAttr = []
new_reb_time = []
new_total_acc = []
temp_demand = 0
for origin in range(duration_data.shape[0]):
    for destination in range(duration_data.shape[1]):
        for ts in range(episode_length):
            attr = defaultdict()
            attr['time_stamp'] = ts
            attr['origin'] = origin
            attr['destination'] = destination
            attr['demand'] = round(od_data[origin,destination,int(ts*time_granularity)]*time_granularity) # create equal distributed demand over granular time 
            temp_demand += attr['demand']
            attr['price'] = distance_data[origin,destination,int(ts*time_granularity)]*0.91 + duration_data[origin,destination,int(ts*time_granularity)]*time_granularity * 0.39 + 2.20 + 2.70 # in $
            new_tripAttr.append(attr)

            reb = defaultdict()
            reb['time_stamp'] = ts
            reb['origin'] = origin
            reb['destination'] = destination
            reb['reb_time'] = int(duration_data[origin,destination,int(ts*time_granularity)])
            new_reb_time.append(reb)
print(temp_demand)
# should be able to change 24 to 1, we don't change the number of vehicles across the day
for hour in range(24):
    acc = defaultdict()
    acc['hour'] = hour
    acc['acc'] = math.ceil(fleet_size)
    new_total_acc.append(acc)
new_data = defaultdict()
new_data['demand'] = new_tripAttr
new_data['rebTime'] = new_reb_time
new_data['totalAcc'] = new_total_acc
new_data['chargelevels'] = number_chargelevels
new_data['spatialNodes'] = number_spatial_nodes
new_data['chargeLevelsPerChargeStep'] = charge_levels_per_charge_step
new_data['episodeLength'] = episode_length
new_data['energy_prices'] = p_energy.tolist()
new_data['chargeLocations'] = charge_locations
new_data['carsPerStationCapacity'] = cars_per_station_capacity
new_data['timeGranularity'] = time_granularity
new_data['operationalCostPerTimestep'] = operational_cost_per_timestep
print(episode_length)
with open('SF_5_short_afternoon.json', 'w') as f:
    json.dump(new_data, f)

1228026
16


In [4]:
new_tripAttr = []
new_reb_time = []
new_total_acc = []
temp_demand = 0
for origin in range(duration_data.shape[0]):
    for destination in range(duration_data.shape[1]):
        for ts in range(episode_length):
            attr = defaultdict()
            attr['time_stamp'] = ts
            attr['origin'] = origin
            attr['destination'] = destination
            attr['demand'] = round(0.95*od_data[origin,destination,int(ts*time_granularity)]*time_granularity) # create equal distributed demand over granular time 
            temp_demand += attr['demand']
            attr['price'] = distance_data[origin,destination,int(ts*time_granularity)]*0.91 + duration_data[origin,destination,int(ts*time_granularity)]*time_granularity * 0.39 + 2.20 + 2.70 # in $
            new_tripAttr.append(attr)

            reb = defaultdict()
            reb['time_stamp'] = ts
            reb['origin'] = origin
            reb['destination'] = destination
            reb['reb_time'] = int(duration_data[origin,destination,int(ts*time_granularity)])
            new_reb_time.append(reb)
print(temp_demand)
# should be able to change 24 to 1, we don't change the number of vehicles across the day
for hour in range(24):
    acc = defaultdict()
    acc['hour'] = hour
    acc['acc'] = math.ceil(fleet_size)
    new_total_acc.append(acc)
new_data = defaultdict()
new_data['demand'] = new_tripAttr
new_data['rebTime'] = new_reb_time
new_data['totalAcc'] = new_total_acc
new_data['chargelevels'] = number_chargelevels
new_data['spatialNodes'] = number_spatial_nodes
new_data['chargeLevelsPerChargeStep'] = charge_levels_per_charge_step
new_data['episodeLength'] = episode_length
new_data['energy_prices'] = p_energy.tolist()
new_data['chargeLocations'] = charge_locations
new_data['carsPerStationCapacity'] = cars_per_station_capacity
new_data['timeGranularity'] = time_granularity
new_data['operationalCostPerTimestep'] = operational_cost_per_timestep
print(episode_length)
with open('SF_5_short_afternoon_test.json', 'w') as f:
    json.dump(new_data, f)

1166638
16


In [5]:
# preprocessing for skrewed data
od_data = np.load('od_matrix.npy')
duration_data = np.load('duration_matrix.npy')
distance_data = np.load('distance_matrix.npy')
charging_stations = np.load('charging_stations.npy')
demand_per_time = np.zeros(od_data.shape[2])
for t in range(od_data.shape[2]):
    demand_per_time[t] = od_data[:,:,t].sum()
low_demand_times = demand_per_time < demand_per_time.max()*0.7
od_data = od_data[:, :, 8:-4] # in requests
duration_data = duration_data[:, :, 8:-4] # in sec
distance_data = distance_data[:, :, 8:-4] # in miles
print(od_data.shape)

# set remaing nans to avg
avg_duration = int(np.nanmean(duration_data))
duration_data[np.isnan(duration_data)] = avg_duration
avg_distance = int(np.nanmean(distance_data))
distance_data[np.isnan(distance_data)] = avg_distance
delta_c = 1.5 # energy step [kWh] 0.75
time_granularity = 0.5 # in h
artificial_decrease_in_demand = 0.5

chevy_bolt_capacity = 65 # in kWh
chevy_bolt_usable_capacity = chevy_bolt_capacity * 0.6 # never go below 20% or above 80% of charge
charger_capacity = 50.0 # assuming 50. kW Chargers
assert (delta_c/charger_capacity)/time_granularity < 1
charge_levels_per_charge_step = math.floor((charger_capacity*time_granularity)/delta_c)
# print(charge_levels_per_charge_step)
chevy_bolt_range = 230 # range in mi for mild city trips according to https://media.chevrolet.com/media/us/en/chevrolet/2022-bolt-euv-bolt-ev.detail.html/content/Pages/news/us/en/2021/feb/0214-boltev-bolteuv-specifications.html
chevy_bolt_usable_range = chevy_bolt_range*0.6 # never go below 20% or above 80% of charge and assume 10% less efficient because of range https://cleantechnica.com/2017/10/13/autonomous-cars-shorter-range-due-high-power-consumption-computers/
chevy_bolt_kwh_per_mi = (chevy_bolt_usable_capacity/chevy_bolt_usable_range)/0.7 # we lose 30% efficiency because of AVs (https://medium.com/@teraki/energy-consumption-required-by-edge-computing-reduces-a-autonomous-cars-mileage-with-up-to-30-46b6764ea1b7)
print((chevy_bolt_kwh_per_mi)/delta_c)
# print(chevy_bolt_kwh_per_mi)
energy_distance = np.ceil(((distance_data * chevy_bolt_kwh_per_mi)/delta_c).max(axis=2))
energy_distance[energy_distance==0] = 1 # we should always use energy to satisfy a trip
print(energy_distance)
np.save('energy_distance_zeros.npy', energy_distance)
# print(np.sum(energy_distance==1.))
# print(energy_distance.max())
duration_data = np.round(duration_data/(3600*time_granularity)) # convert travel time from sec to h
duration_data[duration_data==0] = 1. # it should always take time to satisfy a trip
data_timespan = od_data.shape[2]
episode_length = int(data_timespan/time_granularity)
# https://www.pge.com/tariffs/assets/pdf/tariffbook/ELEC_SCHEDS_BEV.pdf for prices of energy
p_energy = np.ones(episode_length)*0.16872 # in $/kWh
p_energy[int(16/time_granularity):int(21/time_granularity)] = 0.38195 # peak prices
p_energy[int(9/time_granularity):int(14/time_granularity)] = 0.14545 # super off peak prices
p_energy *= delta_c # in $/ charge level
p_travel = 0.0770 # [$ / mi] https://newsroom.aaa.com/wp-content/uploads/2021/08/2021-YDC-Brochure-Live.pdf 
avg_miles_per_h_driving = 9.6 # in m/h from 190x190x24
operational_cost_per_timestep = avg_miles_per_h_driving * time_granularity * p_travel
print(operational_cost_per_timestep)
fleet_size = 116616# *0.75 # got number from Justin: 116616

number_chargelevels = int(chevy_bolt_usable_capacity/delta_c)
number_spatial_nodes = 5
charge_locations = np.ones(number_spatial_nodes,dtype=bool).tolist()
cars_per_station_capacity = charging_stations.tolist()

print(charge_levels_per_charge_step)
print(number_chargelevels)

(5, 5, 12)
[[1. 2. 3. 2. 2.]
 [2. 1. 2. 2. 3.]
 [3. 2. 1. 2. 2.]
 [2. 2. 2. 1. 2.]
 [2. 3. 2. 2. 1.]]
0.3696
16
26


In [6]:
new_tripAttr = []
new_reb_time = []
new_total_acc = []
test_sum =  0
for origin in range(duration_data.shape[0]):
    for destination in range(duration_data.shape[1]):
        for ts in range(episode_length):
            attr = defaultdict()
            attr['time_stamp'] = ts
            attr['origin'] = origin
            attr['destination'] = destination
            if origin < 3:
                attr['demand'] = 2*round(od_data[origin,destination,int(ts*time_granularity)]*time_granularity*artificial_decrease_in_demand) # create equal distributed demand over granular time 
            else:
                attr['demand'] = 0*round(od_data[origin,destination,int(ts*time_granularity)]*time_granularity*artificial_decrease_in_demand) # create equal distributed demand over granular time 
            test_sum += attr['demand']
            attr['price'] = distance_data[origin,destination,int(ts*time_granularity)]*0.91 + duration_data[origin,destination,int(ts*time_granularity)]*time_granularity * 0.39 + 2.20 + 2.70 # in $
            new_tripAttr.append(attr)

            reb = defaultdict()
            reb['time_stamp'] = ts
            reb['origin'] = origin
            reb['destination'] = destination
            reb['reb_time'] = int(duration_data[origin,destination,int(ts*time_granularity)])
            new_reb_time.append(reb)

# should be able to change 24 to 1, we don't change the number of vehicles across the day
print(test_sum)
for hour in range(24):
    acc = defaultdict()
    acc['hour'] = hour
    acc['acc'] = math.ceil(fleet_size)
    new_total_acc.append(acc)
new_data = defaultdict()
new_data['demand'] = new_tripAttr
new_data['rebTime'] = new_reb_time
new_data['totalAcc'] = new_total_acc
new_data['chargelevels'] = number_chargelevels
new_data['spatialNodes'] = number_spatial_nodes
new_data['chargeLevelsPerChargeStep'] = charge_levels_per_charge_step
new_data['episodeLength'] = episode_length
new_data['peakHours'] = [0, 6, 7, 8, 9, 10]
new_data['energy_prices'] = p_energy.tolist()
new_data['chargeLocations'] = charge_locations
new_data['carsPerStationCapacity'] = cars_per_station_capacity
new_data['timeGranularity'] = time_granularity
new_data['operationalCostPerTimestep'] = operational_cost_per_timestep
print(episode_length)
with open('SF_5_zeros.json', 'w') as f:
    json.dump(new_data, f)

1311232
24


In [7]:
new_tripAttr = []
new_reb_time = []
new_total_acc = []
test_sum =  0
for origin in range(duration_data.shape[0]):
    for destination in range(duration_data.shape[1]):
        for ts in range(episode_length):
            attr = defaultdict()
            attr['time_stamp'] = ts
            attr['origin'] = origin
            attr['destination'] = destination
            if origin < 3:
                attr['demand'] = round(0.95*2*od_data[origin,destination,int(ts*time_granularity)]*time_granularity*artificial_decrease_in_demand) # create equal distributed demand over granular time 
            else:
                attr['demand'] = 0*round(od_data[origin,destination,int(ts*time_granularity)]*time_granularity*artificial_decrease_in_demand) # create equal distributed demand over granular time 
            test_sum += attr['demand']
            attr['price'] = distance_data[origin,destination,int(ts*time_granularity)]*0.91 + duration_data[origin,destination,int(ts*time_granularity)]*time_granularity * 0.39 + 2.20 + 2.70 # in $
            new_tripAttr.append(attr)

            reb = defaultdict()
            reb['time_stamp'] = ts
            reb['origin'] = origin
            reb['destination'] = destination
            reb['reb_time'] = int(duration_data[origin,destination,int(ts*time_granularity)])
            new_reb_time.append(reb)

# should be able to change 24 to 1, we don't change the number of vehicles across the day
print(test_sum)
for hour in range(24):
    acc = defaultdict()
    acc['hour'] = hour
    acc['acc'] = math.ceil(fleet_size)
    new_total_acc.append(acc)
new_data = defaultdict()
new_data['demand'] = new_tripAttr
new_data['rebTime'] = new_reb_time
new_data['totalAcc'] = new_total_acc
new_data['chargelevels'] = number_chargelevels
new_data['spatialNodes'] = number_spatial_nodes
new_data['chargeLevelsPerChargeStep'] = charge_levels_per_charge_step
new_data['episodeLength'] = episode_length
new_data['peakHours'] = [0, 6, 7, 8, 9, 10]
new_data['energy_prices'] = p_energy.tolist()
new_data['chargeLocations'] = charge_locations
new_data['carsPerStationCapacity'] = cars_per_station_capacity
new_data['timeGranularity'] = time_granularity
new_data['operationalCostPerTimestep'] = operational_cost_per_timestep
print(episode_length)
with open('SF_5_zeros_test.json', 'w') as f:
    json.dump(new_data, f)

1245658
24
